# NLP Interview Assistant: Combining Knowledge Retrieval with LLMs

## Sudip Das

In [ ]:
import torch
import PyPDF2
from PyPDF2 import PdfReader
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


## Data Collection
### PyPDF (scrape content from PDFs)

In [ ]:
!pip install PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Function to clean question text
def clean_question_text(question):
    question = re.sub(r"^Q\d+\.\s*", "", question).strip()
    return question

# Load and extract text
file_path = "100 NLP interview questions and answers.pdf"
reader = PdfReader(file_path)

# Combine all text from the PDF
all_text = ""
for page in reader.pages:
    all_text += page.extract_text()

# Split text into individual Q&A
qa_pattern = r"Q\d+\..*?(?=Q\d+\.|$)"
qa_matches = re.findall(qa_pattern, all_text, re.DOTALL)

# Function to clean the answer text
def clean_answer_text(answer):
    return re.sub(r"^Answer\s*:\s*", "", answer).strip()

# Parse questions and answers
qa_pairs = []
for qa in qa_matches:
    question_match = re.search(r"(Q\d+\..*?)(Answer\s*:.*)", qa, re.DOTALL)
    if question_match:
        # Clean the question and answer
        question = clean_question_text(question_match.group(1).strip())
        answer = clean_answer_text(question_match.group(2).strip())      # Remove Answer
        qa_pairs.append({"Question": question, "Answer": answer})

# Create a DataFrame for structured processing
qa_df_pdf = pd.DataFrame(qa_pairs)

print(qa_df_pdf.head())

                                            Question  \
0  Which of the following techniques can be used ...   
1  Which of the following techniques can be used ...   
2  What are the possible features of a text corpu...   
3  You created a document term matrix on the inpu...   
4  Which of the text parsing techniques can be us...   

                                              Answer  
0  a) Lemmatization helps to get to the base form...  
1  b) and c) \nDistance between two word vectors ...  
2  e)All of the above can be used as features of ...  
3                                                 d)  
4                                                 d)  


In [ ]:
# Print the first 3 question-answer pairs
for i in range(3):
    print(f"Question {i+1}:")
    print(qa_df_pdf.loc[i, "Question"])
    print("Answer:")
    print(qa_df_pdf.loc[i, "Answer"])
    print("-" * 50)

Question 1:
Which of the following techniques can be used for keyword normalization in 
NLP, the process of converting a keyword into its base form? 
a. Lemmatization 
b. Soundex 
c. Cosine Similarity 
d. N-grams
Answer:
a) Lemmatization helps to get to the base form of a word, e.g. are playing -> play, ea ting 
-> eat, etc.Other options are meant for different purposes.
--------------------------------------------------
Question 2:
Which of the following techniques can be used to compute the distance 
between two word vectors in NLP? 
a. Lemmatization 
b. Euclidean distance 
c. Cosine Similarity 
d. N-grams
Answer:
b) and c) 
Distance between two word vectors can be computed using Cosine similarity and Euclidean 
Distance.  Cosine Similarity establishes a cosine angle between the vector of two words . A cosi ne 
angle close to each other between two word vectors indicates the words are simil ar and vice a 
versa. 
E.g. cosine angle between two words “Football” and “Cricket” will be cl

### Web Scraping (BeautifulSoup)

In [ ]:
# URL of the webpage to scrape
url = "https://www.projectpro.io/article/nlp-interview-questions-and-answers/439"

response = requests.get(url)
if response.status_code == 200:
    print("Successfully fetched the webpage!")
else:
    print(f"Failed to fetch the webpage. Status code: {response.status_code}")

soup = BeautifulSoup(response.content, "html.parser")

def clean_question_text(question):
    question = re.sub(r"^Question \d+:", "", question).strip()
    # Remove any numbering like "1.", "2." at the start of the actual question
    question = re.sub(r"^\d+\.\s*", "", question).strip()
    return question

# Extract Q&A pairs
qa_pairs = []
questions = soup.find_all("p", {"style": "padding-left: 40px;"})  # Match questions based on style

for question in questions:
    # Extract and clean question text
    question_text = question.find("strong")
    if question_text:
        q_text = clean_question_text(question_text.get_text(strip=True))

        answer_paragraphs = []
        sibling = question.find_next_sibling()
        while sibling and sibling.name == "p" and "padding-left: 40px;" not in sibling.get("style", ""):
            if sibling.find("table") is None:
                answer_paragraphs.append(sibling.get_text(strip=True))
            sibling = sibling.find_next_sibling()

        if answer_paragraphs:
            a_text = " ".join(answer_paragraphs)
            qa_pairs.append({"Question": q_text, "Answer": a_text})

web_qa_df = pd.DataFrame(qa_pairs)

print(web_qa_df.head())

Successfully fetched the webpage!
                                            Question  \
0                        What do you know about NLP?   
1  Give examples of any two real-world applicatio...   
2                       What is tokenization in NLP?   
3  What is the difference between stemming and le...   
4                                       What is NLU?   

                                              Answer  
0  NLP stands for Natural Language Processing. It...  
1  1. Spelling/Grammar Checking Apps:The mobile a...  
2  Tokenization is the process of splitting runni...  
3  Both stemming and lemmatization are keyword no...  
4  NLU stands for Natural Language Understanding....  


In [ ]:
# Print the first 3 questions and answers
for i in range(3):
    print(f"Question {i+1}:")
    print(web_qa_df.loc[i, "Question"])
    print("Answer:")
    print(web_qa_df.loc[i, "Answer"])
    print("-" * 50)


Question 1:
What do you know about NLP?
Answer:
NLP stands for Natural Language Processing. It deals with making a machine understand the way human beings read and write in a language. This task is achieved by designing algorithms that can extract meaning from large datasets in audio or text format by applying machine learning algorithms.
--------------------------------------------------
Question 2:
Give examples of any two real-world applications of NLP.
Answer:
1. Spelling/Grammar Checking Apps:The mobile applications and websites that offer users correct grammar mistakes in the entered text rely on NLP algorithms. These days, they can also recommend the following few words that the user might type, which is also because of specific NLP models being used in the backend. 2.ChatBots:Many websites now offer customer support through these virtual bots that chat with the user and resolve their problems. It acts as a filter to the issues that do not require an interaction with the compani

In [ ]:
# Website 2
url = "https://www.adaface.com/blog/nlp-interview-questions/"

response = requests.get(url)
if response.status_code == 200:
    print("Successfully fetched the webpage!")
    page_content = response.text
else:
    print(f"Failed to fetch the webpage. Status code: {response.status_code}")
    exit()

# Parse the webpage content
soup = BeautifulSoup(page_content, 'html.parser')

qa_pairs = []

# questions are in <h3> tags, checked from developer tools
questions = soup.find_all('h3')

for question in questions:
    # Extract the question text
    q_text = question.get_text(strip=True)

    # Remove leading numbers like "1.", "2.", etc.
    q_text = re.sub(r"^\d+\.\s*", "", q_text)

    # Collect all answer elements until the next <h3> tag
    answer_elements = question.find_next_siblings()
    answer_text = ""

    for element in answer_elements:
        # Stop if another question is encountered
        if element.name == "h3":
            break
        if element.name == "p":
            answer_text += element.get_text(strip=True) + " "

    qa_pairs.append({"Question": q_text.strip(), "Answer": answer_text.strip()})

qa_df = pd.DataFrame(qa_pairs)


# Display the first 3 question-answer pairs
for i in range(min(3, len(qa_df))):  # Ensure we don't try to access rows if fewer than 3 exist
    print(f"Question {i+1}:")
    print(qa_df.loc[i, "Question"])
    print("Answer:")
    print(qa_df.loc[i, "Answer"])
    print("-" * 50)



Successfully fetched the webpage!
Question 1:
What is Natural Language Processing (NLP), and why is it important?
Answer:
Natural Language Processing (NLP) is a field of artificial intelligence that focuses on the interaction between computers and humans through natural language. It involves teaching machines to understand, interpret, and generate human language in a way that is valuable. NLP is important because it enables applications like translation services, sentiment analysis, chatbots, and more, which can process large amounts of data quickly and efficiently. It helps in automating routine tasks, improving customer service, and providing insights from unstructured data. When evaluating the candidate’s response, look for a clear and concise explanation, an understanding of practical applications, and the ability to relate NLP to real-world scenarios and benefits.
--------------------------------------------------
Question 2:
Can you explain the difference between NLP and text min

## Data Preprocessing

In [ ]:
# Combine all scraped data into a single DataFrame
qa_df_combined = pd.concat([qa_df_pdf, web_qa_df, qa_df], ignore_index=True)

In [ ]:
# Calculate word count in the combined dataframe
qa_df_combined['Word Count'] = qa_df_combined['Question'].apply(lambda x: len(str(x).split())) + qa_df_combined['Answer'].apply(lambda x: len(str(x).split()))
print(f"Total Word Count in qa_df_combined: {qa_df_combined['Word Count'].sum()}")

Total Word Count in qa_df_combined: 11959


In [ ]:
!pip install sentence_transformers
!pip install faiss-cpu
!pip install faiss-gpu
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM



In [ ]:
# Load a sentence transformer model to encode the questions
os.environ["HF_TOKEN"] = "" #add your HF token
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight embedding model
questions = qa_df_combined["Question"].tolist()

# Convert questions to embeddings
embeddings = embedding_model.encode(questions)

In [ ]:
# Create a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))  # Add embeddings to the index

faiss.write_index(index, "faiss_index.bin") #Save

In [ ]:
# Retrieval Function
def retrieve_context(query, index, questions, model, top_k=3, similarity_threshold=0.7):
    query_embedding = model.encode([query])

    # Search the FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the top-k results
    context = []
    for idx, dist in zip(indices[0], distances[0]):
        if dist < similarity_threshold:
            context.append(questions[idx])

    return context if context else []


## LLMs - Llama-3.2 and Flan-T5

In [ ]:
# Load Llama 3.2 model from hugging face
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
lm_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct").to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM
# Load Flan-T5 model
flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)

In [ ]:
def generate_answer(query, context, tokenizer, model):

    #corpus has mcq questions, hence needs to handle it separately if question asked later is not an mcq and matches with the original question
    is_mcq_context = context and any(opt in '. '.join(context) for opt in ["a.", "b.", "c.", "d."])
    is_mcq_query = any(keyword in query.lower() for keyword in ["choose", "select", "option", "correct"])

    if not context:
        input_text = f"Question: {query}\n\nProvide a brief answer:" #use llm if context not available
    elif is_mcq_context and is_mcq_query:
        # Handle MCQ format only if the query is MCQ
        input_text = f"Context: {'. '.join(context)}\n\nQuestion: {query}\n\nProvide a concise explanation for the correct answer only, ignoring other options:"
    else:
        # Handle normal question format or non-MCQ queries
        input_text = f"Context: {'. '.join(context)}\n\nQuestion: {query}\n\nProvide a short and accurate Answer:"


    # Tokenize and generate
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=400, num_beams=5, early_stopping=True)

    # Decode the answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)


    # Remove MCQ artifacts if the query is not explicitly an MCQ
    if not is_mcq_query:
        answer = re.sub(r"^[A-Da-d]\)\s*", "", answer)
        # truncate the explanation for incorrect options after the correct answer
        answer = re.split(r"Note:|Explanation:", answer)[0].strip()



    # Generic cleanup
    finality_patterns = [
        r"(?i)the best answer is.*",
        r"(?i)therefore, the correct answer is.*",
        r"(?i)correct answer:.*"
    ]
    for pattern in finality_patterns:
        answer = re.sub(pattern, "", answer)


    # Remove the question from the answer (case-insensitive)
    pattern = rf"(?i)Question:\s*{re.escape(query)}?"
    answer = re.sub(pattern, "", answer, flags=re.DOTALL)
    # Remove context from the answer
    context_pattern = rf"(?i)Context:\s*{re.escape('. '.join(context) if context else '')}?"
    answer = re.sub(context_pattern, "", answer, flags=re.DOTALL)

    # Remove generic headers like "Context" or "Provide a brief answer"
    answer = re.sub(r"(?i)context:.*?Question:.*?", "", answer, flags=re.DOTALL)
    answer = re.sub(r"(?i)provide a brief answer:?", "", answer, flags=re.DOTALL)
    answer = re.sub(r"(?i)Provide a concise explanation for the correct answer only, ignoring other optionsr*?:", "", answer)
    answer = re.sub(r"(?i)Provide a short and accurate Answer*?:", "", answer)
    answer = answer.split("Answer:", 1)[-1].strip()

    return answer


In [ ]:
# Test model
query = "What are stop words, and why are they removed in NLP?"
retrieved_context = retrieve_context(query, index, questions, embedding_model)
generated_answer = generate_answer(query, retrieved_context, tokenizer, lm_model)

print(f"Question: {query}")
print(f"Retrieved Context: {retrieved_context}")
print("-" * 80)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What are stop words, and why are they removed in NLP?
Retrieved Context: ['What are stop words, and why are they removed in text processing?', 'What are stop words?', 'In NLP, The process of removing words like “and”, “is”, “a”, “an”, “the” from \na sentence is called as \na. Stemming \nb. Lemmatization \nc. Stop word \nd. All of the above']
--------------------------------------------------------------------------------
Generated Answer: Stop words are common words like “and”, “is”, “a”, “an”, “the” that do not add much value to the meaning of a sentence. They are removed in NLP to reduce the dimensionality of the text data and improve the performance of NLP tasks like text classification, sentiment analysis, and information retrieval.


## Evaluation

In [ ]:
# List of random 30 common questions in NLP
q = [
    "What is Natural Language Processing (NLP)?",
    "Explain the difference between NLP and text mining.",
    "What are stop words, and why are they removed in NLP?",
    "What is tokenization in NLP?",
    "How can I prepare for a Natural Language Processing (NLP) interview?",
    "Which technique can be used for noun phrase detection in NLP?",
    "What is the purpose of Part-of-Speech (POS) tagging in NLP?",
    "How does named entity recognition (NER) work, and what are its applications?",
    "Define Dependency Parsing and its importance in NLP.",
    "What is the Bag-of-Words (BoW) model?",
    "How is TF-IDF used for text representation in NLP?",
    "What is word embedding, and how is it different from one-hot encoding?",
    "Explain the Skip-Gram model in Word2Vec.",
    "How does Transfer Learning improve NLP model performance?",
    "What are the advantages of using pre-trained models in NLP?",
    "How does a Transformer model work in NLP?",
    "What is the significance of the attention mechanism in NLP models?",
    "How do BERT and GPT differ in their approach to NLP tasks?",
    "What is a language model, and why is it important in NLP?",
    "Explain the concept of fine-tuning a pre-trained model for NLP tasks.",
    "How is sentiment analysis performed using NLP techniques?",
    "What are some real-world applications of NLP in healthcare?",
    "How does machine translation work in NLP?",
    "What is text summarization, and how can it be implemented in NLP?",
    "How does question answering work in NLP systems?",
    "What are some common challenges in text preprocessing for NLP?",
    "How does ambiguity in human language affect NLP systems?",
    "What are the limitations of current NLP models?",
    "How does NLP handle multilingual text?",
    "How can NLP models be evaluated for performance and accuracy?"
]


In [ ]:
# Function to store the outputs of the 30 questions
def store_answers(queries, filename, tokenizer, lm_model):
    results = []
    for query in q:
        retrieved_context = retrieve_context(query, index, questions, embedding_model)
        generated_answer = generate_answer(query, retrieved_context, tokenizer, lm_model)
        print(f"Question: {query}")
        #print(f"Retrieved Context: {retrieved_context}")
        print(f"Generated Answer: {generated_answer}")
        print("-" * 50)

        results.append({
            "Question": query,
            "Generated Answer": generated_answer
        })

    # Save results to a JSON file
    with open(filename, "w") as f:
        import json
        json.dump(results, f, indent=4)

### Testing Llama 3.2 response

In [ ]:
store_answers(q, "llama_answers.json", tokenizer, lm_model) #and display

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is Natural Language Processing (NLP)?
Generated Answer: Natural Language Processing (NLP) is a subfield of artificial intelligence (AI) that deals with the interaction between computers and humans in natural language. It involves the development of algorithms and statistical models that enable computers to process, understand, and generate natural language data. NLP is a crucial component of many applications, including speech recognition, language translation, sentiment analysis, and text summarization.

Common elements of NLP:

1.  **Tokenization**: Breaking down text into individual words or tokens.
2.  **Stopword removal**: Removing common words like "the," "and," and "a" that do not add much value to the meaning of the text.
3.  **Stemming or Lemmatization**: Reducing words to their base form (e.g., "running" becomes "run").
4.  **Part-of-speech tagging**: Identifying the grammatical category of each word (e.g., noun, verb, adjective).
5.  **Named entity recognition

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Explain the difference between NLP and text mining.
Generated Answer: NLP (Natural Language Processing) is a subfield of artificial intelligence that deals with the interaction between computers and humans in natural language. It involves the development of algorithms and statistical models that enable computers to process, understand, and generate natural language data. Text mining, on the other hand, is a subset of NLP that focuses specifically on extracting insights and patterns from large volumes of unstructured text data. In other words, NLP is a broader field that encompasses text mining, among other tasks such as sentiment analysis, language translation, and speech recognition.

Key Points:

*   NLP is a subfield of AI that deals with human-computer interaction in natural language.
*   Text mining is a subset of NLP that focuses on extracting insights from unstructured text data.
*   NLP encompasses a range of tasks beyond text mining, including sentiment analysis, lan

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Question: How does ambiguity in human language affect NLP systems?
Generated Answer: Ambiguity in human language can significantly impact NLP systems, as it can lead to misinterpretation and incorrect processing of text. For example, homophones (words with the same pronunciation but different meanings) and homographs (words with the same spelling but different meanings) can cause confusion. Additionally, context-dependent words and idioms can be challenging for NLP systems to understand. To mitigate these issues, NLP systems can employ techniques such as part-of-speech tagging, named entity recognition, and semantic role labeling to disambiguate language and improve accuracy.
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What are the limitations of current NLP models?
Generated Answer: Current NLP models face several limitations, including:

1.  **Lack of common sense**: NLP models often struggle to understand the nuances of human language, such as idioms, sarcasm, and figurative language.
2.  **Limited contextual understanding**: NLP models may not fully comprehend the context in which a piece of text is being used, leading to misinterpretations.
3.  **Inability to generalize**: NLP models are often trained on specific datasets and may not generalize well to new, unseen data.
4.  **Vulnerability to bias**: NLP models can perpetuate biases present in the training data, leading to unfair or discriminatory outcomes.
5.  **Inadequate handling of ambiguity**: NLP models may struggle to handle ambiguous or unclear language, leading to incorrect interpretations.

These limitations highlight the need for continued research and development in NLP to improve the accuracy and reliability of NLP models.

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: How does NLP handle multilingual text?
Generated Answer: ## Step 1: Understanding NLP Basics
NLP (Natural Language Processing) is a subfield of AI that deals with the interaction between computers and humans in natural language. It involves tasks such as text processing, sentiment analysis, and machine translation.

## Step 2: Handling Multilingual Text
To handle multilingual text in NLP, several approaches can be employed. These include:

- **Tokenization**: Breaking down text into individual words or tokens, regardless of language.
- **Stemming or Lemmatization**: Reducing words to their base form to facilitate comparison across languages.
- **Machine Translation**: Using machine learning models to translate text from one language to another.
- **Language Detection**: Identifying the language of the text to apply the most suitable processing techniques.
- **Multilingual Models**: Training models that can handle multiple languages simultaneously.

## Step 3: Key Consideratio

### Testing Flan-T5 response

In [ ]:
store_answers(q, "T5_answers.json",flan_tokenizer, flan_model)

Question: What is Natural Language Processing (NLP)?
Generated Answer: Natural Language Processing (NLP) is a branch of computer science that deals with the processing of natural language.
--------------------------------------------------
Question: Explain the difference between NLP and text mining.
Generated Answer: Natural Language Processing (NLP) is a branch of computer science that focuses on the processing of natural language.
--------------------------------------------------
Question: What are stop words, and why are they removed in NLP?
Generated Answer: In NLP, The process of removing words like “and”, “is”, “a”, “an”, “the” from a sentence is called as a. Stemming b. Lemmatization c. Stop word d.
--------------------------------------------------
Question: What is tokenization in NLP?
Generated Answer: Tokenization is the process of separating a set of words from a set of words.
--------------------------------------------------
Question: How can I prepare for a Natural Lan

In [7]:
!pip install evaluate
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=15781feef968465afb914cd80b559167ad4ecefefbe7e32d29363f9573fb1204
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [8]:
import json
import evaluate

def load_answers_from_file(filepath):

    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    answers = [item["Generated Answer"] for item in data]
    return answers

# File paths for reference and model answer files
reference_file = ('/content/drive/My Drive/references.json')
llama_file = ('/content/drive/My Drive/llama_answers.json')
t5_file = ('/content/drive/My Drive/T5_answers.json')

# Load the answers
reference_answers = load_answers_from_file(reference_file)
llama_answers = load_answers_from_file(llama_file)
t5_answers = load_answers_from_file(t5_file)

# Validate that all lists have the same length
if not (len(reference_answers) == len(llama_answers) == len(t5_answers)):
    raise ValueError("The number of answers in reference, llama, and t5 files must match.")

# Load evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Prepare references for BLEU (as BLEU expects a list of lists for references)
references_for_bleu = [[ref] for ref in reference_answers]

# Compute BLEU for each model
bleu_llama = bleu.compute(predictions=llama_answers, references=references_for_bleu)
bleu_t5 = bleu.compute(predictions=t5_answers, references=references_for_bleu)

# Compute ROUGE for each model
rouge_llama = rouge.compute(predictions=llama_answers, references=reference_answers)
rouge_t5 = rouge.compute(predictions=t5_answers, references=reference_answers)

# Print the results
print("=== Model Evaluation Results ===\n")

print("--- BLEU Scores ---")
print(f"LLaMA BLEU: {bleu_llama}")
print(f"T5 BLEU: {bleu_t5}")

print("\n--- ROUGE Scores ---")
print("LLaMA ROUGE:", rouge_llama)
print("T5 ROUGE:", rouge_t5)

# Simple summary comparison based on BLEU and ROUGE-L scores
bleu_llama_score = bleu_llama["bleu"]
bleu_t5_score = bleu_t5["bleu"]

rouge_llama_l = rouge_llama["rougeL"]
rouge_t5_l = rouge_t5["rougeL"]

print("\n--- Summary ---")
if bleu_llama_score > bleu_t5_score:
    print("LLaMA performed better on BLEU.")
else:
    print("T5 performed better on BLEU.")

if rouge_llama_l > rouge_t5_l:
    print("LLaMA performed better on ROUGE-L.")
else:
    print("T5 performed better on ROUGE-L.")

=== Model Evaluation Results ===

--- BLEU Scores ---
LLaMA BLEU: {'bleu': 0.030191903818260386, 'precisions': [0.12669100279149667, 0.03911821914847633, 0.01740265390472047, 0.00963433326034596], 'brevity_penalty': 1.0, 'length_ratio': 4.583661417322834, 'translation_length': 4657, 'reference_length': 1016}
T5 BLEU: {'bleu': 0.021804534750438788, 'precisions': [0.33405639913232105, 0.08120649651972157, 0.04239401496259352, 0.02425876010781671], 'brevity_penalty': 0.3000204754021258, 'length_ratio': 0.453740157480315, 'translation_length': 461, 'reference_length': 1016}

--- ROUGE Scores ---
LLaMA ROUGE: {'rouge1': 0.24456696186268784, 'rouge2': 0.08778528539099956, 'rougeL': 0.18828350598392585, 'rougeLsum': 0.2019726109482999}
T5 ROUGE: {'rouge1': 0.2134161650600847, 'rouge2': 0.075212847308192, 'rougeL': 0.17982557010379158, 'rougeLsum': 0.17963872543855225}

--- Summary ---
LLaMA performed better on BLEU.
LLaMA performed better on ROUGE-L.


## Test your knowledge

In [ ]:
import random

def get_random_mcq_question(qa_df_combined):
    # Filter rows that are likely to be MCQs (contain "a.", "b.", etc.)
    mcq_questions = qa_df_combined[qa_df_combined['Question'].str.contains(r'\ba\.', regex=True, na=False)]

    if mcq_questions.empty:
        return "No MCQ questions found in the corpus."

    # Randomly select one MCQ
    mcq = mcq_questions.sample(n=1).iloc[0]
    question = mcq['Question']
    answer = mcq['Answer']

    return question, answer

# Example usage
question, answer = get_random_mcq_question(qa_df_combined)
print(f"Question: {question}")  # Displays the question

Question: In NLP, The process of converting a sentence or paragraph into tokens is 
referred to as Stemming 
a. True 
b. False


In [ ]:
# Display the answer
answer

'b) The statement describes the process of tokenization and not stemming, hence it  is \nFalse.'

In [ ]:
# Test model
query = "What are stop words, and why are they removed in NLP?"
retrieved_context = retrieve_context(query, index, questions, embedding_model)
generated_answer = generate_answer(query, retrieved_context, tokenizer, lm_model)

print(f"Question: {query}")
print(f"Retrieved Context: {retrieved_context}")
print("-" * 80)
print(f"Generated Answer: {generated_answer}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What are stop words, and why are they removed in NLP?
Retrieved Context: ['What are stop words, and why are they removed in text processing?', 'What are stop words?', 'In NLP, The process of removing words like “and”, “is”, “a”, “an”, “the” from \na sentence is called as \na. Stemming \nb. Lemmatization \nc. Stop word \nd. All of the above']
--------------------------------------------------------------------------------
Generated Answer: Stop words are common words like “and”, “is”, “a”, “an”, “the” that do not add much value to the meaning of a sentence. They are removed in NLP to reduce the dimensionality of the text data and improve the performance of NLP tasks like text classification, sentiment analysis, and information retrieval.


## UI using Gradio

In [ ]:
!pip install gradio
import gradio as gr

def chatbot(model_choice, query):
    if model_choice == "Llama_3.2":
        retrieved_context = retrieve_context(query, index, questions, embedding_model)
        return generate_answer(query, retrieved_context, tokenizer, lm_model)
    elif model_choice == "Flan-T5":
        retrieved_context = retrieve_context(query, index, questions, embedding_model)
        return generate_answer(query, retrieved_context, flan_tokenizer, flan_model)

# Define the Gradio UI
def gradio_ui():
    model_choices = ["Llama_3.2", "Flan-T5"]

    with gr.Blocks() as demo:
        gr.Markdown("# NLP Interview Assistant")

        with gr.Row():
            model_selector = gr.Dropdown(choices=model_choices, label="Select Model", value="Llama_3.2")

        query_input = gr.Textbox(label="Enter your query", placeholder="Type your query here...", lines=2)

        output_box = gr.Textbox(label="Model Output", placeholder="Response will appear here", lines=4)

        submit_button = gr.Button("Submit")

        # Link the chatbot function to the Gradio UI
        submit_button.click(chatbot, inputs=[model_selector, query_input], outputs=output_box)

    return demo

# Create the Gradio UI
demo = gradio_ui()

# Launch the UI
demo.launch(share=True)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://598e98d5bb0023c320.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
